In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path
import datetime
import sys
import json

In [ ]:
features_json_data = open('./features.json').read()
features = json.loads(features_json_data)

In [ ]:
from data_extraction import *

In [ ]:
stays = getICUStayPatients()

In [ ]:
stays.head(10)

In [ ]:
getPatientTimeSeries(stays[stays['icustay_id']==200021].iloc[0])

In [ ]:
stays['intime'][0]

In [ ]:
getPatientTimeSeries(stays.iloc[0])

In [ ]:
getPatientTimeSeries(stays.iloc[11575])

In [ ]:
allTimeSeries = normalizeFeatures(None)

In [ ]:
len(allTimeSeries)

In [ ]:
allTimeSeries[0].columns

In [ ]:
indices = set()
for i, ts in enumerate(allTimeSeries):
    maxValue = ts['1 hours urine output'].max()
    minValue = ts['1 hours urine output'].min()
    if maxValue > 0.5 or minValue < 0.03:
        print(i, maxValue, minValue, ts['hadm_id'].iloc[0])
        indices.add(i)

In [ ]:
allTimeSeries = [timeSeries for i, timeSeries in enumerate(allTimeSeries) if i not in indices]
feature = '1 hours urine output'
allTimeSeriesConcat = pd.concat(allTimeSeries)
minValue = allTimeSeriesConcat[feature].min()
maxValue = allTimeSeriesConcat[feature].max()
diff = maxValue - minValue
allTimeSeriesConcat[feature] = (allTimeSeriesConcat[feature]-minValue)/diff
allTimeSeriesConcat.to_csv('time_series_normalized.csv', index=False)
allTimeSeries = [timeSeries.sort_values('Time') for icustayID, timeSeries in
                     allTimeSeriesConcat.groupby('icustay_id')]

In [ ]:
allTimeSeries[8].head(20)

In [ ]:
allTimeSeriesConcat = pd.concat(allTimeSeries)

In [ ]:
feature = 'Heart Rate'
minValue = allTimeSeriesConcat[feature].min()
maxValue = allTimeSeriesConcat[feature].max()
diff = maxValue - minValue
allTimeSeriesConcat[feature] = (allTimeSeriesConcat[feature]-minValue)/diff

In [ ]:
allTimeSeriesConcat['Heart Rate']

In [ ]:
allTimeSeries[0].columns

In [ ]:
features = ['age', 'gender', 'Heart Rate',
       'Respiratory Rate', 'SpO2/SaO2', 'pH', 'Potassium', 'Calcium',
       'Glucose', 'Sodium', 'HCO3', 'White Blood Cells', 'Hemoglobin',
       'Red Blood Cells', 'Platelet Count', 'Weight', 'Urea Nitrogen',
       'Creatinine', 'Blood Pressure', 'gcs',
       'ventilation', 'vasoactive medications', 'sedative medications', '1 hours urine output', '6 hours urine output', 'AKI']

In [ ]:
allTimeSeries = [timeSeries[features] for timeSeries in allTimeSeries]

In [ ]:
len(allTimeSeries[20].columns)

In [ ]:
allTimeSeriesConcat = pd.concat(allTimeSeries)
ratio = len(allTimeSeriesConcat)/allTimeSeriesConcat['AKI'].sum()
ratio

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
train, test = train_test_split(allTimeSeries, test_size=0.2, random_state=12)

In [ ]:
from data_iterator import *

In [ ]:
iterator = DataIterator(train)

In [ ]:
x, y, sequenceLengths, musks = iterator.next_batch(256)

In [ ]:
x.shape

In [ ]:
import tensorflow as tf

In [ ]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [ ]:
trainGraph(graph, sess, train, test)

In [ ]:
import multi_layer_lstm

In [ ]:
tf.reset_default_graph()

In [ ]:
graph = multi_layer_lstm.buildGraph(stateSizes=(512, 512), numFeatures=25)

In [ ]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [ ]:
multi_layer_lstm.trainGraph(graph, sess, train, test)

In [ ]:
feed = {graph['x']: x, graph['y']: y, graph['seqlen']: sequenceLengths, graph['musk']: musks}

In [ ]:
sess.run(graph['cost'], feed_dict=feed)

In [ ]:
tf.reset_default_graph()

In [ ]:
import self_correcting_gru

In [ ]:
graph = self_correcting_gru.buildGraph()

In [ ]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [ ]:
self_correcting_gru.trainGraph(graph, sess, train, test)